In [4]:
#Tic tac toe
from random import choice

DICT_SYMBOLES = {0:" ", -1:"o", 1:"x"}

KEYB_FR = [["A", "Z", "E"], 
           ["Q", "S", "D"], 
           ["W", "X", "C"]
          ]

KEYB_US = [["Q", "W", "E"], 
           ["A", "S", "D"], 
           ["Z", "X", "C"]
          ]

KEYB_BP = [["B", "É", "P"], 
           ["A", "U", "I"], 
           ["À", "Y", "X"]
          ]

COMMANDES_NUM = [["7", "8", "9"], 
                 ["4", "5", "6"], 
                 ["1", "2", "3"]
                ]

COMMANDES_ALPHA = KEYB_FR


# lorsque l'ordinateur joue, vérifie si des lignes sont à compléter/bloquer
def completude_lignes(nature, plateau):
    for ligne in range(3):
        if sum(plateau[ligne]) == (nature * 2):
            for colonne in range(3):
                if plateau[ligne][colonne] == 0:
                    return ligne, colonne
    return -1, -1

# lorsque l'ordinateur joue, permet de transposer le plateau afin d'utiliser "completude_lignes()" pour les colonnes    
def transposer(plateau):
    return [[ligne[i] for ligne in plateau] for i in range(len(plateau[0]))]

# lorsque l'ordinateur jour, vérifie si des diagonales sont à compléter/bloquer
def completude_diagonales(nature, plateau):
    for colonne in (0, 2):
        coo_diagonales = [(0, colonne), (1, 1), (2, 2 - colonne)]
        val_diagonales = [plateau[ligne][colonne] for ligne, colonne in coo_diagonales]
        if sum(val_diagonales) == (2 * nature):
            for i in range(len(val_diagonales)):
                if val_diagonales[i] == 0:
                    ligne, colonne = coo_diagonales[i]
                    return ligne, colonne
    return -1, -1    
    
# algorithme utilisé pour retourner les coordonnées du coup de l'ordinateur
def coup_cpu(cpu, plateau):
    ligne, colonne = -1, -1
    
    # d'abord : compléter une ligne/colonne/diagonale
    if ligne == -1:
        ligne, colonne = completude_lignes(cpu, plateau)
    if ligne == -1:
        colonne, ligne = completude_lignes(cpu, transposer(plateau))   
    if ligne == -1:
        ligne, colonne = completude_diagonales(cpu, plateau) 

    # sinon : bloquer une ligne/colonne/diagonale de l'adversaire
    if ligne == -1:
        ligne, colonne = completude_lignes(-cpu, plateau)
    if ligne == -1:
        colonne, ligne = completude_lignes(-cpu, transposer(plateau))
    if ligne == -1:
        ligne, colonne = completude_diagonales(-cpu, plateau)

    # sinon : jouer le milieu
    if ligne == -1:
        if plateau[1][1] == 0:
            ligne, colonne = 1, 1

    # sinon : jouer un coin
    if ligne == -1:
        coo_coins = [(0,0), (0,2), (2,0), (2,2)]
        val_coins = [plateau[ligne][colonne] for ligne, colonne in coo_coins]
        coins_vides = [coo_coins[i] for i in range(len(val_coins)) if val_coins[i] == 0]
        if len(coins_vides) != 0:
            ligne, colonne = choice(coins_vides)

    # sinon : jouer un côté
    if ligne == -1:
        coo_cotes = [(0,1), (1,0), (1,2), (2,1)]
        val_cotes = [plateau[ligne][colonne] for ligne, colonne in coo_cotes]
        cotes_vides = [coo_cotes[i] for i in range(len(val_cotes)) if val_cotes[i] == 0]
        ligne, colonne = choice(cotes_vides)
        
    return(ligne, colonne)

# affichage du plateau
def afficher_plateau(plateau):
    valeurs = [DICT_SYMBOLES[val] for ligne in plateau for val in ligne]
    ligne_haut = "┌───┬───┬───┐"
    ligne_mid = "├───┼───┼───┤"
    ligne_bas = "└───┴───┴───┘"
    ligne_val1 = f"│ {valeurs[0]} │ {valeurs[1]} │ {valeurs[2]} │"
    ligne_val2 = f"│ {valeurs[3]} │ {valeurs[4]} │ {valeurs[5]} │"
    ligne_val3 = f"│ {valeurs[6]} │ {valeurs[7]} │ {valeurs[8]} │"

    plateau_graphique = [ligne_haut, 
                         ligne_val1, 
                         ligne_mid, 
                         ligne_val2, 
                         ligne_mid, 
                         ligne_val3, 
                         ligne_bas
                        ]

    for ligne in plateau_graphique:
        print(ligne)

# demande le coup du joueur et retourne les coordonnées
def demander_coup(joueur, plateau):
    # liste des coups possibles (cases vides)
    coups_possibles = [valeur for coo_ligne, ligne in enumerate(COMMANDES_NUM) for coo_valeur, valeur in enumerate(ligne) if plateau[coo_ligne][coo_valeur] == 0] + [valeur for coo_ligne, ligne in enumerate(COMMANDES_ALPHA) for coo_valeur, valeur in enumerate(ligne) if plateau[coo_ligne][coo_valeur] == 0]

    # demander dans quelle case le joueur souhaite jouer
    coup = ""
    while coup not in coups_possibles:
        coup = input(f"Où souhaitez-vous jouer '{DICT_SYMBOLES[joueur]}' ? ").upper()
    [(lig, col)] = [(coo_ligne, coo_valeur) for coo_ligne, ligne in enumerate(COMMANDES_NUM) for coo_valeur, valeur in enumerate(ligne) if valeur == coup] + [(coo_ligne, coo_valeur) for coo_ligne, ligne in enumerate(COMMANDES_ALPHA) for coo_valeur, valeur in enumerate(ligne) if valeur == coup]

    # positionnement du coup sur le plateau

    return lig, col

# vérifie si le jeu est terminé (trois symboles identiques alignés ou grille pleine)
def fin_du_jeu(plateau):
    somme_lig = [sum(ligne) for ligne in plateau]
    somme_col = [sum([plateau[ligne][colonne] for ligne in range(len(plateau))]) for colonne in range(len(plateau[0]))]
    somme_diag = [plateau[0][0] + plateau[1][1] + plateau[2][2], plateau[0][2] + plateau[1][1] + plateau[2][0]]
    somme_all = somme_lig + somme_col + somme_diag
    if max(somme_all) == 3:
        print(f"Le joueur '{DICT_SYMBOLES[1]}' gagne !")
        return True
    if min(somme_all) == -3:
        print(f"Le joueur '{DICT_SYMBOLES[-1]}' gagne !")
        return True
    if len([valeur for ligne in plateau for valeur in ligne if valeur == 0]) == 0:
        print("Égalité.")
        return True
    return False

def tictactoe():
    #initialisation des variables
    plateau = [[0, 0, 0], 
               [0, 0, 0], 
               [0, 0, 0]
              ]

    cpu = 0
    joueur = 1

    nb_joueurs = ""
    while nb_joueurs not in ("1", "2"):
        nb_joueurs = input("1 ou 2 joueurs ? : ")

    if nb_joueurs == "1":
        cpu = choice([-1, 1])
    print(cpu)

    while not fin_du_jeu(plateau):
        if joueur == cpu:
            ligne, colonne = coup_cpu(cpu, plateau)
        else:
            ligne, colonne = demander_coup(joueur, plateau)
        plateau[ligne][colonne] = joueur
        afficher_plateau(plateau)
        joueur = - joueur

tictactoe()

1 ou 2 joueurs ? :  1


1
┌───┬───┬───┐
│   │   │   │
├───┼───┼───┤
│   │ x │   │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘


Où souhaitez-vous jouer 'o' ?  7


┌───┬───┬───┐
│ o │   │   │
├───┼───┼───┤
│   │ x │   │
├───┼───┼───┤
│   │   │   │
└───┴───┴───┘
┌───┬───┬───┐
│ o │   │   │
├───┼───┼───┤
│   │ x │   │
├───┼───┼───┤
│   │   │ x │
└───┴───┴───┘


Où souhaitez-vous jouer 'o' ?  1


┌───┬───┬───┐
│ o │   │   │
├───┼───┼───┤
│   │ x │   │
├───┼───┼───┤
│ o │   │ x │
└───┴───┴───┘
┌───┬───┬───┐
│ o │   │   │
├───┼───┼───┤
│ x │ x │   │
├───┼───┼───┤
│ o │   │ x │
└───┴───┴───┘


Où souhaitez-vous jouer 'o' ?  9


┌───┬───┬───┐
│ o │   │ o │
├───┼───┼───┤
│ x │ x │   │
├───┼───┼───┤
│ o │   │ x │
└───┴───┴───┘
┌───┬───┬───┐
│ o │   │ o │
├───┼───┼───┤
│ x │ x │ x │
├───┼───┼───┤
│ o │   │ x │
└───┴───┴───┘
Le joueur 'x' gagne !


In [104]:
[sum([plateau[ligne][colonne] for ligne in range(len(plateau))]) for colonne in range(len(plateau[0]))]

[1, 0, 0]

In [106]:
[plateau[0][0] + plateau[1][1] + plateau[2][2], plateau[0][2] + plateau[1][1] + plateau[2][0]]

[0, 0]

In [118]:
cpu = -1
plateau = [[-1, 0, 0],[0, 1, 0],[0, 0, -1]]

from random import choice


def completude_lignes(nature, plateau):
    for ligne in range(3):
        if sum(plateau[ligne]) == (nature * 2):
            for colonne in range(3):
                if plateau[ligne][colonne] == 0:
                    return ligne, colonne
    return -1, -1
                

def completude_diagonales(nature, plateau):
    for colonne in (0, 2):
        coo_diagonales = [(0, colonne), (1, 1), (2, 2 - colonne)]
        val_diagonales = [plateau[ligne][colonne] for ligne, colonne in coo_diagonales]
        if sum(val_diagonales) == (2 * nature):
            for i in range(len(val_diagonales)):
                if val_diagonales[i] == 0:
                    ligne, colonne = coo_diagonales[i]
                    return ligne, colonne
    return -1, -1    
    
    
def transposer(plateau):
    return [[ligne[i] for ligne in plateau] for i in range(len(plateau[0]))]

def coup_cpu():
    ligne, colonne = -1, -1
    if ligne == -1:
        ligne, colonne = completude_lignes(cpu, plateau)
    if ligne == -1:
        colonne, ligne = completude_lignes(cpu, transposer(plateau))   
    if ligne == -1:
        ligne, colonne = completude_diagonales(cpu, plateau) 

    if ligne == -1:
        ligne, colonne = completude_lignes(-cpu, plateau)
    if ligne == -1:
        colonne, ligne = completude_lignes(-cpu, transposer(plateau))
    if ligne == -1:
        ligne, colonne = completude_diagonales(-cpu, plateau)

    if ligne == -1:
        if plateau[1][1] == 0:
            ligne, colonne = 1, 1

    if ligne == -1:
        coo_coins = [(0,0), (0,2), (2,0), (2,2)]
        val_coins = [plateau[ligne][colonne] for ligne, colonne in coo_coins]
        coins_vides = [coo_coins[i] for i in range(len(val_coins)) if val_coins[i] == 0]
        ligne, colonne = choice(coins_vides)
    
    if ligne == -1:
        coo_cotes = [(0,1), (1,0), (1,2), (2,1)]
        val_coins = [plateau[ligne][colonne] for ligne, colonne in coo_cotes]
        cotes_vides = [coo_cotes[i] for i in range(len(val_cotes)) if val_cotes[i] == 0]
        ligne, colonne = choice(cotes_vides)
        
    return(ligne, colonne)

coup_cpu()

(2, 0)

In [112]:
transposer(plateau)

[[-1, 0, 0], [0, 1, 0], [0, 0, -1]]

In [113]:
plateau = [[1, 1, 0],[-1, 1, 0],[0, 0, 1]]
[[ligne[i] for ligne in plateau] for i in range(len(plateau[0]))]



[[1, -1, 0], [1, 1, 0], [0, 0, 1]]

In [114]:
plateau = [[-1, 1, 0],[-1, 0, 0],[0, 0, 0]]
[[ligne[i] for i in range(len(ligne))] for ligne in plateau]

[[-1, 1, 0], [-1, 0, 0], [0, 0, 0]]

In [79]:
plateau = [[0, 1, -1],[0, -1, 0],[0, 0, 0]]

nature = -1

def completude_diagonales(nature, plateau):
    for colonne in (0, 2):
        coo_diagonales = [(0, colonne), (1, 1), (2, 2 - colonne)]
        val_diagonales = [plateau[ligne][colonne] for ligne, colonne in coo_diagonales]
        if sum(val_diagonales) == 2 * nature:
            for i in range(len(val_diagonales)):
                if val_diagonales[i] == 0:
                    ligne, colonne = coo_diagonales[i]
                    return ligne, colonne
    return -1, -1

2 0


In [67]:
    coo_diagonales = [(0, colonne), (1, 1), (2, 2 - colonne)]
    val_diagonales = [plateau[ligne][colonne] for ligne, colonne in coo_diagonales]
    print (coo_diagonales, val_diagonales)

[(0, 0), (1, 1), (2, 2)] [-1, -1, 0]


In [7]:
choice([1, 2, 3])

2